# GENERATION OF SAMPLING PATTERN AND NOISE DISTRIBUTIONS POOLS FROM THE ML SUBSAMPLE

In this notebook we generate the pools of sampling patterns and noise distributions from the ML subsample of CARMENES RV curves that we will use to modify the basic synthetic dataset to get a modeled dataset as similar as possible to the real ML subsample dataset.

## Modules and configuration

### Modules

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import json

from IPython.display import clear_output

from scipy.stats import pearsonr, ttest_ind, normaltest, kstest

from distfit import distfit

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white", {'figure.figsize':(15,10)})

### Configuration

In [2]:
GTO_FILE = "../data/SELECTION_for_ML_CARM_VIS_objects_with_PG.csv"
DIST_FILES_FOLDER = "../data/DIST_FILES/"
NOISE_DIST_FOLDER = "../data/DIST_FILES/RV_ML_subsample_Stars_noise_dist/"

RV_PATTERN_POOL_FILE = "RV_ML_subsample_sampling_patterns_and_noise_B_REPEAT.csv"

IMAGE_FOLDER = "./img/"

RANDOM_STATE = 11 # For reproducibility

N_TRIALS = 10 # Number of random noise samples to generate to test the goodness of fit.
ALPHA = 1e-3 # Significance threshold for statistical tests

### Functions

In [3]:
def noise_fit_goodness(source_data, fitted_dist, num_trials, src_min=None, src_max=None):
    '''Returns a median of the p-value for a given number of T-test independence values,
    calculated between a source data and asample from a distribution fitted to that
    source data. Also return a median of the generated values that fall outside the range
    of the source data.'''
    
    # Code to silence output is from Alex Martelli and Nick T at:
    # https://stackoverflow.com/questions/2828953/silence-the-stdout-of-a-function-in-python-without-trashing-sys-stdout-and-resto
    # Redirect output to a file
    #save_stdout = sys.stdout
    #sys.stdout = open('trash', 'w')

    p_values = []
    oor_values = []
    if src_min is None:
        src_min = source_data.min()
    if src_max is None:
        src_max = source_data.max()
    for trial in range(0, num_trials):
        clear_output(wait=True)
        rnd_values = fitted_dist.generate(n=len(source_data), random_state=RANDOM_STATE)
        #_, p = ttest_ind(source_data, rnd_values)
        #oor_fraction = (sum(rnd_values < src_min) + sum(rnd_values > src_max)) / len(rnd_values)
        #p_values.append(p)
        #oor_values.append(oor_fraction)
        oor_fraction = (sum(rnd_values < src_min) + sum(rnd_values > src_max)) / len(rnd_values)
        oor_values.append(oor_fraction)
        #_, p = ttest_ind(source_data, rnd_values)
        _, ks_p = kstest(source_data, rnd_values)
        p_values.append(ks_p)
        print("ks_p %f" %ks_p)
    
    # Redirect output to screen again, and delete the file.
    #sys.stdout = save_stdout
    #os.remove('trash')
    
    return np.nanmedian(p_values), np.nanmedian(oor_values)
    

In [4]:
def rv_extract_sampling_and_noise(rv_filename):
    '''Reads a CARMENES RV curve file and returns the sampling pattern as a list of time differences,
    referred to the time of the initial sample point, and the best fit for the noise distribution,
    as well as noise statistics and estimated goodness of fit for the noise'''
    if True: # TEST
    #try:
        rv_ts = pd.read_csv(rv_filename, sep=' ', decimal='.', names=['time', 'rv', 'rv_error'])
        # Extract sampling pattern:
        ref_time = rv_ts['time'].min()
        sampling_pattern = (rv_ts['time'] - ref_time).tolist()
        # Calculate noise statistics:
        rv_error_median = np.nanmedian(rv_ts['rv_error'])
        rv_error_mean = np.nanmean(rv_ts['rv_error'])
        rv_error_stdev = np.nanstd(rv_ts['rv_error'])
        rv_error_min = np.nanmin(rv_ts['rv_error'])
        rv_error_max = np.nanmax(rv_ts['rv_error'])
        # Fit best noise distribution:
        dist = distfit()
        dist.fit_transform(pd.Series(rv_ts['rv_error']), verbose=0)
        if True: # TEST
        #try:
            fit_p_value, fit_outliers_fraction = \
                noise_fit_goodness(source_data=rv_ts['rv_error'], fitted_dist=dist, num_trials=N_TRIALS,
                                   src_min=rv_error_min, src_max=rv_error_max)
        else: # TEST
        #except:
            fit_p_value, fit_outliers_fraction = 0.0, np.nan
    else: # TEST
    #except Exception as e:
        print("***ERROR: an error happened with file %s. Error: %s" %(rv_filename, str(e)))
        sampling_pattern = None
        fit_p_value = 0.0
    return sampling_pattern, rv_error_min, rv_error_max, rv_error_median, rv_error_mean, rv_error_stdev, \
        dist, fit_p_value, fit_outliers_fraction

In [5]:
def draw_boxplot(data, x, y, fig_title, x_label, y_label, showfliers=None, fig_filename=None):
    '''Draws a boxplot according to the arguments passed.'''
    # Plots and saves the figure
    #kwargs = dict(hist_kws={'alpha':.4}, kde_kws={'linewidth':2})
    plt.figure(figsize=(10,7))
    sns.boxplot(data=data, x=x, y=y, showfliers=showfliers)
    plt.title(fig_title, fontsize=16)
    plt.xlabel(x_label, fontsize=12)
    plt.ylabel(y_label, fontsize=12)
    #plt.grid(axis='both', alpha=0.75)
    if fig_filename is None:
        pass
    else:
        plt.savefig(fig_filename, format='png')


In [6]:
def scatter_graph(data, var_x, var_y, fig_title, x_label, y_label, fig_filename=None):
    '''Plots a scatter plot of two columns of a dataframe.
    It also calculates and shows Pearson's correlation coefficient.'''
    #rv_x_var = var_x
    #tess_y_var = var_y
    #kwargs = dict(hist_kws={'alpha':.4}, kde_kws={'linewidth':2})
    plt.figure(figsize=(10,7))
    plt.title(fig_title, fontsize=16)
    plt.grid(axis='both', alpha=0.75)
    plt.xlabel(x_label, fontsize=12)
    plt.ylabel(y_label, fontsize=12)
    sns.scatterplot(data=data, x=var_x, y=var_y)
#    plt.xlim(0.0, 1.0)
#    plt.ylim(0.0, 1.0)    
    plt.legend(loc='upper right');
    # Save the image:
    if fig_filename is None:
        pass
    else:
        plt.savefig(fig_filename, format='png')
    # Calculate the Pearson correlation coefficient between the two results:
    corr, _ = pearsonr(data[var_x], data[var_y])
    print('Pearson\'s correlation value: %.3f' % corr)

In [7]:
def draw_hist(d1, d1_label, fig_filename=None):
    '''Draws an histogram of the values passed. It also calulates the statistics
    and tests the goodness of fit by a Gaussian distribution'''
    # Plots and saves the figure
    kwargs = dict(hist_kws={'alpha':.4}, kde_kws={'linewidth':2})
    plt.figure(figsize=(10,7))
    plt.title(d1_label + " values distribution", fontsize=16)
    plt.grid(axis='both', alpha=0.75)
    plt.xlabel(d1_label, fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    sns.distplot(d1, label=d1_label, **kwargs)
    #sns.histplot(d1, label=d1_label)
    #plt.xlim(0,100)
    #plt.legend(loc='upper right');
    # Save the image:
    if fig_filename is None:
        pass
    else:
        plt.savefig(fig_filename, format='png')
    # Display basic statistics:
    print(pd.Series(d1).describe())
    # Perform the normaltest (D'Agostino and Pearson) for Gaussian distribution fit:
    k2, p = normaltest(d1)
    alpha = ALPHA
    print("p = {:g}".format(p))
    if p < alpha:  # null hypothesis: x comes from a normal distribution
        print("***This distribution does NOT come from a Gaussian distribution" \
              "(with significance level = %.6f)" %(1.0 - alpha))
    else:
        print("This distribution MIGHT COME come from Gaussian distribution" \
              "(with significance level = %.6f)" %(1.0 - alpha))

## Load the ML subsample file

In [8]:
gto = pd.read_csv(GTO_FILE, sep=',', decimal='.')
gto.head()

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,WF_offset_PG_TESS,WF_e_offset_PG_TESS,WF_FAP_PG_TESS,WF_valid_PG_TESS,WF_error_PG_TESS,WF_elapsed_time_PG_TESS,WF_plain_file_TESS,WF_fig_file_TESS,PG_file_RV,PG_file_TESS
0,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1000.000122,9.022946e-07,1.0,1.0,NaN,132.607176,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23505-09...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...,../data/CARM_VIS_RVs_PGs/J23505-095_RV_PG.dat,../data/CARM_VIS_TESS_PGs/J23505-095_RV_PG.dat
1,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001995,NaN,NaN,../data/CARM_VIS_RVs_PGs/J23492+024_RV_PG.dat,NaN
2,J23431+365,GJ 1289,-,1289,355.781509,36.536310,23:43:06.31,+36:32:13.1,107.922839,-24.336479,...,999.999512,4.306074e-06,1.0,1.0,NaN,97.939914,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23431+36...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...,../data/CARM_VIS_RVs_PGs/J23431+365_RV_PG.dat,../data/CARM_VIS_TESS_PGs/J23431+365_RV_PG.dat
3,J23419+441,HH And,-,905,355.480015,44.170376,23:41:55.04,+44:10:38.8,109.989338,-16.947350,...,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000998,NaN,NaN,../data/CARM_VIS_RVs_PGs/J23419+441_RV_PG.dat,NaN
4,J23381-162,G 273-093,-,4352,354.532687,-16.236514,23:38:08.16,-16:14:10.2,61.845437,-69.825220,...,1000.000122,9.022946e-07,1.0,1.0,NaN,136.603404,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23381-16...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...,../data/CARM_VIS_RVs_PGs/J23381-162_RV_PG.dat,../data/CARM_VIS_TESS_PGs/J23381-162_RV_PG.dat


### Clean the object list

To be on the safe side, we remove the objects that gave trouble during the RV periodogram calculations:

In [9]:
gto.loc[gto['valid_PG_RV'] == 0, 'Karmn']

Series([], Name: Karmn, dtype: object)

In [10]:
gto = gto[gto['valid_PG_RV'] == 1].reset_index().copy()
gto.head()

,index,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,...,WF_offset_PG_TESS,WF_e_offset_PG_TESS,WF_FAP_PG_TESS,WF_valid_PG_TESS,WF_error_PG_TESS,WF_elapsed_time_PG_TESS,WF_plain_file_TESS,WF_fig_file_TESS,PG_file_RV,PG_file_TESS
0,0,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,...,1000.000122,9.022946e-07,1.0,1.0,NaN,132.607176,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23505-09...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...,../data/CARM_VIS_RVs_PGs/J23505-095_RV_PG.dat,../data/CARM_VIS_TESS_PGs/J23505-095_RV_PG.dat
1,1,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,...,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001995,NaN,NaN,../data/CARM_VIS_RVs_PGs/J23492+024_RV_PG.dat,NaN
2,2,J23431+365,GJ 1289,-,1289,355.781509,36.536310,23:43:06.31,+36:32:13.1,107.922839,...,999.999512,4.306074e-06,1.0,1.0,NaN,97.939914,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23431+36...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...,../data/CARM_VIS_RVs_PGs/J23431+365_RV_PG.dat,../data/CARM_VIS_TESS_PGs/J23431+365_RV_PG.dat
3,3,J23419+441,HH And,-,905,355.480015,44.170376,23:41:55.04,+44:10:38.8,109.989338,...,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000998,NaN,NaN,../data/CARM_VIS_RVs_PGs/J23419+441_RV_PG.dat,NaN
4,4,J23381-162,G 273-093,-,4352,354.532687,-16.236514,23:38:08.16,-16:14:10.2,61.845437,...,1000.000122,9.022946e-07,1.0,1.0,NaN,136.603404,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23381-16...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...,../data/CARM_VIS_RVs_PGs/J23381-162_RV_PG.dat,../data/CARM_VIS_TESS_PGs/J23381-162_RV_PG.dat


In [11]:
gto.shape

(233, 301)

## CARMENES ML subsample RV sampling patterns and noise distributions pool

### Populate the pools

Notice that we also trasfer some of the original data from the star, so that calculating some possible correlations later on will be easier.

In [12]:
print(gto.columns.to_list())

['index', 'Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', '

In [13]:
print(gto['Activity'].unique().tolist())

[nan, 'Flare?', 'Flare']


In [14]:
sum(gto['Activity'].isna())

205

In [15]:
# Initialize the results array
rv_patterns = pd.DataFrame(columns=['Karmn', 'Vr_kms-1', 'n_RV', 'Ps_RV', 'wrms_RV',
                                    'RV_sampling_deltas',
                                    'RV_noise_min', 'RV_noise_max',
                                    'RV_noise_median', 'RV_noise_mean', 'RV_noise_stdev',
                                    'RV_noise_dist', 'RV_noise_name', 'RV_noise_loc', 'RV_noise_scale',
                                    'RV_noise_dist_file', 'RV_noise_fit_p-value', 'Fraction_Out'])
rv_patterns

,Karmn,Vr_kms-1,n_RV,Ps_RV,wrms_RV,RV_sampling_deltas,RV_noise_min,RV_noise_max,RV_noise_median,RV_noise_mean,RV_noise_stdev,RV_noise_dist,RV_noise_name,RV_noise_loc,RV_noise_scale,RV_noise_dist_file,RV_noise_fit_p-value,Fraction_Out


In [16]:
# Populate the patterns:
#for i in range(0,5): # TEST
for i in range(0,len(gto)):
    #clear_output(wait=True)
    new_pattern, \
        lower_limit, higher_limit, noise_median, noise_mean, noise_stdev, \
        new_noise_dist, new_noise_fit_p, limit_outliers_frac = \
            rv_extract_sampling_and_noise(gto.loc[i, 'rv_file'])
    dist_filename = NOISE_DIST_FOLDER + gto.loc[i, 'Karmn'] + "_RV_noise.pickle"
    new_noise_dist.save(dist_filename)
    try:
        new_noise_name = new_noise_dist.model['name']
    except:
        new_noise_name = np.nan
    try:
        new_noise_loc = new_noise_dist.model['loc']
    except:
        new_noise_loc = np.nan
    try:
        new_noise_scale = new_noise_dist.model['scale']
    except:
        new_noise_scale = np.nan
    rv_patterns = rv_patterns \
        .append({'Karmn': gto.loc[i, 'Karmn'],
                 'Vr_kms-1': gto.loc[i, 'Vr_kms-1'],
                 'n_RV': gto.loc[i, 'n_RV'],
                 'Ps_RV': gto.loc[i, 'Ps_RV'],
                 'wrms_RV': gto.loc[i, 'wrms_RV'],
                 'RV_sampling_deltas': json.dumps(new_pattern),
                 'RV_noise_min': lower_limit,
                 'RV_noise_max': higher_limit,
                 'RV_noise_median': noise_median,
                 'RV_noise_mean': noise_mean,
                 'RV_noise_stdev': noise_stdev,
                 'RV_noise_dist': new_noise_dist.model,
                 'RV_noise_name': new_noise_name,                 
                 'RV_noise_loc': new_noise_loc,
                 'RV_noise_scale':  new_noise_scale,
                 'RV_noise_dist_file': dist_filename,
                 'RV_noise_fit_p-value': new_noise_fit_p,
                 'Fraction_Out': limit_outliers_frac
                },
                ignore_index=True)
    #print("noise p-value %f" %new_noise_fit_p)
    
#rv_patterns.tail()

[distfit] >Generate 52 lognorm distributed samples with fitted params (0.6439650964457941, 0.9073017392164592, 0.7272116932075658).
ks_p 0.420624
[pypickle] Pickle file saved: [../data/DIST_FILES/RV_ML_subsample_Stars_noise_dist/J00051+457_RV_noise.pickle]
[distfit] >Saving.. True


In [17]:
rv_patterns.tail()

,Karmn,Vr_kms-1,n_RV,Ps_RV,wrms_RV,RV_sampling_deltas,RV_noise_min,RV_noise_max,RV_noise_median,RV_noise_mean,RV_noise_stdev,RV_noise_dist,RV_noise_name,RV_noise_loc,RV_noise_scale,RV_noise_dist_file,RV_noise_fit_p-value,Fraction_Out
228,J00184+440,10.997000,193.0,0.985929,2.311629,"[0.0, 13.990557200275362, 18.001077729277313, ...",0.832274,3.104075,1.331101,1.535932,0.554882,{'distr': <scipy.stats._continuous_distns.dwei...,dweibull,1.240303,3.236921e-01,../data/DIST_FILES/RV_ML_subsample_Stars_noise...,0.003439,0.124352
229,J00183+440,11.820000,216.0,0.826708,2.617721,"[0.0, 1.0492656417191029, 2.022110936231911, 3...",0.766382,4.264498,1.476962,1.556480,0.527647,{'distr': <scipy.stats._continuous_distns.beta...,beta,0.744054,7.930984e+00,../data/DIST_FILES/RV_ML_subsample_Stars_noise...,0.313226,0.000000
230,J00162+198E,-1.625194,18.0,42.940403,2.861697,"[0.0, 15.97568494733423, 18.95287090772763, 18...",1.102225,2.145801,1.483359,1.577230,0.338830,{'distr': <scipy.stats._continuous_distns.beta...,beta,1.102225,1.076078e+00,../data/DIST_FILES/RV_ML_subsample_Stars_noise...,0.132394,0.000000
231,J00067-075,-40.123002,89.0,2.510671,2.440083,"[0.0, 0.9991974416188896, 11.033571805804968, ...",1.113746,5.291221,1.482946,1.572339,0.508195,{'distr': <scipy.stats._continuous_distns.beta...,beta,1.090486,1.083765e+13,../data/DIST_FILES/RV_ML_subsample_Stars_noise...,0.395307,0.000000
232,J00051+457,-0.229000,52.0,3.024201,2.835825,"[0.0, 20.975221153348684, 188.3961285338737, 2...",1.053913,5.406914,1.706992,1.804720,0.676688,{'distr': <scipy.stats._continuous_distns.logn...,lognorm,0.907302,7.272117e-01,../data/DIST_FILES/RV_ML_subsample_Stars_noise...,0.420624,0.019231


#### Check the p-values obtained

In [18]:
rv_patterns[['RV_noise_fit_p-value', 'Fraction_Out']].describe()

,RV_noise_fit_p-value,Fraction_Out
count,2.330000e+02,233.000000
mean,6.896301e-01,0.064660
std,2.748911e-01,0.101262
min,2.543246e-10,0.000000
25%,5.025901e-01,0.000000
50%,7.695470e-01,0.016667
75%,9.184050e-01,0.102767
max,1.000000e+00,0.714286


In [19]:
rv_patterns.head()

,Karmn,Vr_kms-1,n_RV,Ps_RV,wrms_RV,RV_sampling_deltas,RV_noise_min,RV_noise_max,RV_noise_median,RV_noise_mean,RV_noise_stdev,RV_noise_dist,RV_noise_name,RV_noise_loc,RV_noise_scale,RV_noise_dist_file,RV_noise_fit_p-value,Fraction_Out
0,J23505-095,-21.870867,71.0,2.908164,3.362607,"[0.0, 11.957513123750687, 47.87879143701866, 5...",1.199479,4.170413,1.722917,1.881812,0.628702,{'distr': <scipy.stats._continuous_distns.t_ge...,t,1.691561,0.290193,../data/DIST_FILES/RV_ML_subsample_Stars_noise...,0.762093,0.112676
1,J23492+024,-71.131000,373.0,1.143762,2.183867,"[0.0, 17.0164248123765, 190.38824744801968, 21...",0.951985,7.585994,1.517377,1.656111,0.599510,{'distr': <scipy.stats._continuous_distns.logn...,lognorm,0.862271,0.660178,../data/DIST_FILES/RV_ML_subsample_Stars_noise...,0.883145,0.000000
2,J23431+365,-2.597000,23.0,17.057200,2.795865,"[0.0, 8.942367421463132, 9.875712749548256, 25...",0.916620,2.010952,1.377055,1.408319,0.262370,{'distr': <scipy.stats._continuous_distns.dwei...,dweibull,1.403768,0.233191,../data/DIST_FILES/RV_ML_subsample_Stars_noise...,0.888037,0.043478
3,J23419+441,-77.764000,99.0,3.031857,2.532704,"[0.0, 136.37014811998233, 154.37618444254622, ...",0.829277,2.496016,1.297172,1.363447,0.307447,{'distr': <scipy.stats._continuous_distns.gene...,genextreme,1.223698,0.212595,../data/DIST_FILES/RV_ML_subsample_Stars_noise...,0.696110,0.000000
4,J23381-162,20.470749,56.0,3.019729,2.405056,"[0.0, 10.978803387377411, 30.955036497674882, ...",1.016080,3.446221,1.423674,1.611703,0.542434,{'distr': <scipy.stats._continuous_distns.gene...,genextreme,1.343269,0.248643,../data/DIST_FILES/RV_ML_subsample_Stars_noise...,0.778950,0.017857


In [20]:
# See the last pattern added:
print(new_pattern)

[0.0, 20.975221153348684, 188.3961285338737, 205.38048223266378, 209.38376312609762, 210.38946344563738, 211.3372044339776, 220.15363332116976, 233.3020677617751, 236.28708868660033, 245.30540515342727, 247.20802798401564, 248.3613522797823, 249.32451374921948, 253.2377212671563, 257.244304608088, 278.1298939133994, 285.19921019487083, 288.1815115036443, 295.1172859589569, 305.06492951279506, 311.06623519863933, 510.30484426533803, 536.3358177728951, 537.3314265487716, 539.2210110668093, 546.3397195651196, 551.2010640129447, 554.1786011150107, 556.2090220963582, 560.3323087557219, 561.3038359247148, 562.3306985800155, 564.3055038368329, 565.3272510636598, 568.3514519101009, 570.3849403518252, 571.3390916050412, 577.1731422855519, 587.3009799011052, 597.2429402219132, 600.2649239646271, 601.2674847017042, 602.3093053563498, 605.2399070998654, 607.2822220451199, 609.2366422573105, 630.1764049907215, 658.1371545102447, 659.1520783104934, 660.1032314212061, 693.2164790052921]


### Save the pools to a file

In [21]:
# Save the file:
rv_patterns.to_csv(DIST_FILES_FOLDER + RV_PATTERN_POOL_FILE, sep=',', decimal='.', index=False)

### Test the access to the saved file and patterns

In [23]:
# Check file re-reading:
reloaded_patterns = pd.read_csv(DIST_FILES_FOLDER + RV_PATTERN_POOL_FILE, sep=',', decimal='.')
reloaded_patterns.tail()

,Karmn,Vr_kms-1,n_RV,Ps_RV,wrms_RV,RV_sampling_deltas,RV_noise_min,RV_noise_max,RV_noise_median,RV_noise_mean,RV_noise_stdev,RV_noise_dist,RV_noise_name,RV_noise_loc,RV_noise_scale,RV_noise_dist_file,RV_noise_fit_p-value,Fraction_Out
228,J00184+440,10.997000,193.0,0.985929,2.311629,"[0.0, 13.990557200275362, 18.001077729277313, ...",0.832274,3.104075,1.331101,1.535932,0.554882,{'distr': <scipy.stats._continuous_distns.dwei...,dweibull,1.240303,3.236921e-01,../data/DIST_FILES/RV_ML_subsample_Stars_noise...,0.003439,0.124352
229,J00183+440,11.820000,216.0,0.826708,2.617721,"[0.0, 1.0492656417191029, 2.022110936231911, 3...",0.766382,4.264498,1.476962,1.556480,0.527647,{'distr': <scipy.stats._continuous_distns.beta...,beta,0.744054,7.930984e+00,../data/DIST_FILES/RV_ML_subsample_Stars_noise...,0.313226,0.000000
230,J00162+198E,-1.625194,18.0,42.940403,2.861697,"[0.0, 15.97568494733423, 18.95287090772763, 18...",1.102225,2.145801,1.483359,1.577230,0.338830,{'distr': <scipy.stats._continuous_distns.beta...,beta,1.102225,1.076078e+00,../data/DIST_FILES/RV_ML_subsample_Stars_noise...,0.132394,0.000000
231,J00067-075,-40.123002,89.0,2.510671,2.440083,"[0.0, 0.9991974416188896, 11.033571805804968, ...",1.113746,5.291221,1.482946,1.572339,0.508195,{'distr': <scipy.stats._continuous_distns.beta...,beta,1.090486,1.083765e+13,../data/DIST_FILES/RV_ML_subsample_Stars_noise...,0.395307,0.000000
232,J00051+457,-0.229000,52.0,3.024201,2.835825,"[0.0, 20.975221153348684, 188.3961285338737, 2...",1.053913,5.406914,1.706992,1.804720,0.676688,{'distr': <scipy.stats._continuous_distns.logn...,lognorm,0.907302,7.272117e-01,../data/DIST_FILES/RV_ML_subsample_Stars_noise...,0.420624,0.019231


In [24]:
# Check pattern access:
recovered_pattern = reloaded_patterns.loc[150, 'RV_sampling_deltas']
recovered_pattern

'[0.0, 22.929542809259146, 48.787756943143904, 49.868371705990285, 50.849283705465496, 51.89953195117414]'

In [25]:
# Notice that we need to convert this string to an array:
rec_pattern_array = json.loads(recovered_pattern)
print(rec_pattern_array)

[0.0, 22.929542809259146, 48.787756943143904, 49.868371705990285, 50.849283705465496, 51.89953195117414]


In [26]:
type(rec_pattern_array)

list

In [27]:
# Check access to a given pattern position:
rec_pattern_array[4]

50.849283705465496

## Summary

**CONCLUSIONS:**
- We have generated a pool of sampling patterns for all ML subsample CARMENES RV curves that will be later used to affect the benchmark synthetic database with irregular sampling in a random way.